import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Finding the Coordinates of Players Birthplaces

# Imports 

In [9]:
import pandas as pd
from functions import *
%run functions.py
import geopy
from geopy.geocoders import Nominatim

In [3]:
player_info_df = pd.read_csv("data/player_info_df.csv")
player_info_df.drop(columns = ['Unnamed: 0', 'index'], inplace=True)

In [4]:
n_am_players = player_info_df[player_info_df['nationality'].isin(['CAN', 'USA'])].reset_index(drop=True)
can_players = n_am_players[n_am_players['birthCountry'] == 'CAN'].reset_index(drop=True)
us_players = n_am_players[n_am_players['birthCountry'] == 'USA'].reset_index(drop=True)

# Using Geopy

In [11]:
# initializing geopy
geolocator = Nominatim(user_agent="NHL_birthplace_analysis")

In [18]:
# the locations that I will feed into geopy
n_am_players['location'] = n_am_players['birthCity'] + "," + n_am_players['birthStateProvince']

In [22]:
# running the function to find all locations
# this took slightly long but was still faster than using wikipedia
lat = []
lng = []
for item in n_am_players['location']:
    try:
        location = geolocator.geocode(item)
        lat.append(location.latitude)
        lng.append(location.longitude)
    except AttributeError:
        lat.append(None)
        lng.append(None)

In [24]:
n_am_players.shape[0] == len(lat)

True

In [25]:
# so only 2 values couldn't be found 
lat.count(None)

2

## Correcting the errors


In [35]:
# player 1
index_1 = lat.index(None)
n_am_players.iloc[index_1]['location']

'St-Anne-de-Bellevue,QC'

In [28]:
test_str = turn_coords_to_correct_format("45.4033° N, 73.9504° W")
test_str

'45.4033,-73.9504'

In [39]:
index_2 = lat.index(None, index_1+1)
n_am_players.iloc[index_2]['location']

'Hilton Head Isl.,SC'

In [33]:
test_str = turn_coords_to_correct_format("32.2163° N, 80.7526° W")
test_str

'32.2163,-80.7526'

In [40]:
# adding those values to the coordinates lists
lat[index_1] = 45.4033
lat[index_2] = 32.2163
lng[index_1] = -73.9504
lng[index_2] = -80.7526

## Adding Data to the dataframe

In [41]:
n_am_players['lat'] = lat
n_am_players['lng'] = lng

# Saving Data

In [44]:
n_am_players.to_csv("data/n_am_players_w_coordinates.csv")